In [4]:
#Imports
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np
import pandas as pd
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)
import os
from dotenv import load_dotenv
load_dotenv()
import requests
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from pathlib import Path
%matplotlib inline
import alpaca_trade_api as tradeapi
#import plotly.plotly as py
import plotly.graph_objs as go

In [6]:
csvpath = Path("Resources/Data_ETFs_to_use.csv")
QQQ = pd.read_csv(csvpath)

In [67]:
# Setting axis and cleaning data
QQQ['Date'] = pd.to_datetime(QQQ['Date'], utc=True)
QQQ.set_axis(QQQ['Date'], inplace=True)
QQQ.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)

In [68]:
#Establishing test and train data
close_data = QQQ['close'].values
close_data = close_data.reshape((-1,1))

In [69]:
split_percent = 0.80
split = int(split_percent*len(close_data))

In [70]:
close_train = close_data[:split]
close_test = close_data[split:]

In [71]:
date_train = QQQ['Date'][:split]
date_test = QQQ['Date'][split:]
print(len(close_train))
print(len(close_test))

1954
489


In [76]:
look_back = 90

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [77]:
#Build Model
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')





In [78]:
# Show the model summary
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 10)                480       
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 11        
Total params: 491
Trainable params: 491
Non-trainable params: 0
_________________________________________________________________


In [79]:
# Train the model
num_epochs = 250
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/250
94/94 [==============================] - 4s 25ms/step - loss: 11390.0529
Epoch 2/250
94/94 [==============================] - 2s 25ms/step - loss: 12067.4177
Epoch 3/250
94/94 [==============================] - 2s 26ms/step - loss: nan
Epoch 4/250
94/94 [==============================] - 2s 25ms/step - loss: nan 0s - loss: n
Epoch 5/250
94/94 [==============================] - 2s 26ms/step - loss: nan
Epoch 6/250
94/94 [==============================] - 3s 27ms/step - loss: nan
Epoch 7/250
94/94 [==============================] - 3s 29ms/step - loss: nan
Epoch 8/250
94/94 [==============================] - 3s 29ms/step - loss: nan
Epoch 9/250
94/94 [==============================] - 3s 30ms/step - loss: nan
Epoch 10/250
94/94 [==============================] - 3s 30ms/step - loss: nan
Epoch 11/250
94/94 [==============================] - 3s 29ms/step - loss: nan
Epoch 12/250
94/94 [==============================] - 3s 29ms/step - loss: nan
Epoch 13/250
94/94 [==============

KeyboardInterrupt: 

In [57]:
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "QQQ Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [58]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = QQQ['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [59]:
forecast

array([316.43      , 314.79544067, 314.55618286, 314.42016602,
       314.19873047, 313.9559021 , 313.99945068, 313.76266479,
       314.28747559, 314.41101074, 314.4425354 , 314.35687256,
       314.55288696, 314.34799194, 314.36416626, 314.46221924,
       314.45098877, 314.4196167 , 314.42340088, 314.38165283,
       314.38253784, 314.36004639, 314.36825562, 314.3868103 ,
       314.41488647, 314.42230225, 314.43734741, 314.44869995,
       314.44927979, 314.45358276, 314.46282959])